# Initialisation

In [724]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#%matplotlib tk #pour Linux

# Importer fichier

In [725]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [726]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [727]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

1


# Traitement d'image

In [728]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [729]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = 60

Please click on the point you want to select.
Selected point: (441.9025974025975, 567.551948051948)


In [730]:
def crop(img, x, y, crop_size):
    x_start = int(x - crop_size // 2)
    x_end = int(x + crop_size // 2)
    y_start = int(y - crop_size // 2)
    y_end = int(y + crop_size // 2)

    return img[y_start:y_end, x_start:x_end]

# Fit gaussien sur le point sélectionné

In [731]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [732]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [733]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - crop_sze/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [734]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [735]:
def next_frame(frame_i):
    frame_i += 1
    tiff.seek(frame_i)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_i += 1
        tiff.seek(frame_i)
        original_image = np.array(tiff)

    return [frame_i, original_image]

Débogueur

In [736]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Faire le crop et fit

In [ ]:
def particle_tracker(image, x, y):
    image = denoise(image)
    #print(positions[frame_index][0],positions[frame_index][1])

    cropped_img = crop(image, x, y, crop_sze)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')
    #if frame_index>8:
    #    visionneur(cropped_img)

    nouveau_x = x - crop_sze // 2 + max_coords[1]
    nouveau_y = y - crop_sze // 2 + max_coords[0]

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze)    # Re-crop autour d'une seule particule

    #if frame_index>8:
    #    visionneur(second_crop)


    result_fit = localisateur_gaussien(second_crop, [x, y])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'Positions finales de l index {frame_index}: ({x_new}, {y_new}) ')
    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [ ]:
position_list = [(x,y)]
sigma_list = []
crop_frames = []
big_frames = []

for _ in range(27 - 1):
    print(f'frame index: {frame_index} et len(positions): {len(position_list)}')
    data = particle_tracker(img, position_list, frame_index)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

frame index: 1 et len(positions): 2
frame index: 2 et len(positions): 3
frame index: 3 et len(positions): 4
frame index: 4 et len(positions): 5
frame index: 5 et len(positions): 6
frame index: 6 et len(positions): 7
frame index: 7 et len(positions): 8
frame index: 8 et len(positions): 9
frame index: 9 et len(positions): 10
frame index: 10 et len(positions): 11
frame index: 11 et len(positions): 12
frame index: 12 et len(positions): 13
frame index: 13 et len(positions): 14
frame index: 14 et len(positions): 15
frame index: 15 et len(positions): 16
frame index: 16 et len(positions): 17
frame index: 17 et len(positions): 18
frame index: 18 et len(positions): 19
frame index: 19 et len(positions): 20
frame index: 20 et len(positions): 21
frame index: 21 et len(positions): 22
frame index: 22 et len(positions): 23
frame index: 23 et len(positions): 24
frame index: 24 et len(positions): 25
frame index: 25 et len(positions): 26
frame index: 26 et len(positions): 27


# Résultats

In [739]:
position_list

[(441.9025974025975, 567.551948051948),
 (441.9025974025975, 567.551948051948),
 (441.5872714824088, 568.5913168655402),
 (441.2407357431485, 567.7405687180908),
 (441.2152557489715, 568.9325811006646),
 (441.4673673118123, 569.3407716993431),
 (441.9374028302384, 569.3606436889775),
 (441.74379141782276, 570.0539052368566),
 (441.616175454853, 571.3094452452121),
 (441.7313177847689, 570.0113240564632),
 (442.0142281329039, 571.049460834359),
 (442.87402700720736, 573.3073044258916),
 (442.5430988722535, 573.2811529943147),
 (441.85200654161935, 573.5775964188261),
 (440.9094038622981, 573.6505209644198),
 (441.87161336982786, 578.9985824771967),
 (441.84823725402, 579.7334846445752),
 (442.1501661165205, 579.8617786578852),
 (442.90248597508696, 580.1847602968634),
 (443.08696981196607, 579.3524263880744),
 (442.58102087952915, 580.503881026763),
 (442.4632463549858, 580.6867390745042),
 (442.47386508136225, 581.2792277022494),
 (443.12295304479204, 582.0272075271115),
 (442.97486559

In [740]:
sigma_list

[(3.270192918749122, 4.431043381239054),
 (3.0979480431155832, 4.810740694666396),
 (2.8831378310239457, 4.475510479030111),
 (2.616603939117088, 4.729771306006793),
 (2.802100552617834, 4.663950255091089),
 (2.568253605428795, 4.0285793726748444),
 (2.471514270604141, 4.152259541658073),
 (2.9319420045507365, 4.070628381006426),
 (2.700670206288887, 4.2127177518819705),
 (3.0624039939963636, 4.644773809831006),
 (2.8619425705644974, 4.390351154122787),
 (2.8997495695960533, 4.370676042885759),
 (3.4689114130485543, 4.645606006696235),
 (3.216146557675548, 5.103353603075927),
 (2.852139404756468, 4.6724468676730835),
 (2.9209605167300645, 4.236550790195615),
 (2.9266789513322204, 4.520172242747669),
 (2.7725322875695713, 4.397371916795202),
 (2.7410707225813664, 4.899395941306493),
 (3.116833629494117, 4.6441268315830815),
 (2.703607018194674, 4.4226581560628695),
 (2.913579154413197, 4.8052843483714085),
 (3.096148961915269, 4.437438191479247),
 (3.0476152376350827, 4.729363026949236)

In [741]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

## anim toutes les particules

In [742]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [743]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()